# Libraries, Settings and Imports

In [6]:
import pandas as pd
import numpy as np

import seaborn as sns

import pycountry
import functools
import calendar
from datetime import date

In [7]:
# fix the jupyter autocomplete issue
%config Completer.use_jedi = False
# adjust pandas settings to not display integers in scientific notation
pd.options.display.float_format = '{:.2f}'.format

In [8]:
# read in data and set column names
df = pd.read_csv("data/1.txt", sep=':', header=None, names=["phone", "id", "f_name", "surname", "sex", "current_city", "hometown", "rel_status", "work", "join_date", "email", "dob", "13", "14"], low_memory=False)

# Feature Extraction

## id

In [9]:
# drop the few rows without corresponding ids
df = df.dropna(subset=['id'])
# set the id column to integer type
df['id'] = df['id'].astype('int')
# sort the dataframe by id
df = df.sort_values('id')
# drop rows where there are duplicate ids (caused by multiple phone numbers associated with one account)
df = df.drop_duplicates(subset='id', keep='first')
# set the id column as the index
df = df.set_index('id')

## phone

In [10]:
# extract the country code from the phone number
df['country_code'] = df['phone'].apply(lambda x: str(x)[0:3])
# remove the swedish country codes, which appear to be included with this irish data in error
df = df.loc[df['country_code'] != '467']
# extract the phone provider prefixes
df['phone_prefix'] = df['phone'].apply(lambda x: '0' + str(x)[3:5])
# drop the raw phone number and country code data
df = df.drop(['phone', 'country_code'], axis=1)

## f_name

In [11]:
# no edits

## surname

In [12]:
# no edits

## sex

In [13]:
# clean up some errors in the sex data
df['sex'] = df['sex'].replace({'male (':'male'})

## current_city

In [14]:
def country_extractor(x):
    """
    This function removes all but the rightmost segment of each address.
    """
    
    if pd.notna(x):
        if ',' in x:
            return x.split(', ')[-1]
        else:
            return x
    else:
        return np.nan

In [15]:
@functools.lru_cache(None)
def country_cleaner(x):
    """
    This function attempts to match the rightmost segment of each address to a specific country, using the pycountry library.
    This allows for standardisation for mapping purposes.
    """
    
    try:
        result = pycountry.countries.search_fuzzy(x)
    except Exception:
        return np.nan
    else:
        return result[0].name

In [16]:
def city_extractor(x):
    """
    This function removes all but the leftmost segment of each address.
    """
    
    if pd.notna(x):
        if ',' in x:
            return x.split(', ')[0]
        else:
            return x
    else:
        return np.nan

In [18]:
# extract only the leftmost section of each address
df['current_country'] = df['current_city'].apply(lambda x: country_extractor(x))
# attempt to match each leftmost address section to an actual country
df['current_country'] = df['current_country'].apply(lambda x: country_cleaner(x))
# extract only the rightmost section of each address
df['current_city'] = df['current_city'].apply(lambda x: city_extractor(x))

KeyboardInterrupt: 

## hometown

In [ ]:
# extract only the leftmost section of each address
df['hometown_country'] = df['hometown'].apply(lambda x: country_extractor(x))
# attempt to match each leftmost address section to an actual country
df['hometown_country'] = df['hometown_country'].apply(lambda x: country_cleaner(x))
# remove ireland from the hometown_country data, to exclusively look at foreign countries
df['hometown_country'] = df['hometown_country'].replace({'Ireland': np.nan})
# extract only the rightmost section of each address
df['hometown'] = df['hometown'].apply(lambda x: city_extractor(x))

## relationship_status

In [ ]:
# no edits

## work

In [ ]:
# clean up and standardise some of the work data
df['work'] = df['work'].replace(
    {
        'Health Service Executive': 'HSE',
        'Dublin, Ireland': np.nan,
        'Self employed': 'Self-Employed',
        'Self Employed (Business)': 'Self-Employed',
        'HSE West': 'HSE',
        'Tesco Ireland': 'Tesco',
        'Irish Defence Forces': 'Óglaigh na hÉireann / Irish Defence Forces',
        'None': np.nan,
        'HSE Ireland': 'HSE'
    }
)

## join-date

In [ ]:
def join_date_extractor(x):
    """
    This function simply extracts the date from the datetime data.
    """
    
    if pd.notna(x):
        return x.split(' ')[0]
    else:
        return x

In [ ]:
# extract the join date from the datetime data
df['join_date'] = df['join_date'].apply(lambda x: join_date_extractor(x))
# correct for some dirty join_date data
df['join_date'] = df['join_date'].replace({'1/1/0001': np.nan})
# convert join_date column to datetime
df['join_date'] = pd.to_datetime(df['join_date'])

## email

In [ ]:
# correct for some dirty data
df['email'] = df['email'].replace({0:np.nan, '00':np.nan})
# correct for an error arising from inconsistencies in available data, which caused the import to place some data in varying columns
df['email'] = df['email'].combine_first(df['13'])

In [ ]:
def email_provider_extractor(x):
    """
    This function extracts the email domain from the email column.
    """
    
    if pd.notna(x):
        return x.split('@')[1]
    else:
        return x

In [ ]:
# extract the email provider from the email data
df['email_provider'] = df['email'].apply(lambda x: email_provider_extractor(x))
# drop the raw email data
df.drop('email', axis=1, inplace=True)

## dob

In [ ]:
# correct for some dirty data
df['dob'] = df['dob'].replace({'00 AM':np.nan})
# correct for an error arising from inconsistencies in available data, which caused the import to place some data in varying columns
df['dob'] = df['dob'].combine_first(df['14'])

In [ ]:
def birth_month_extractor(x):
    """
    This function extracts the birth month from the dob column.
    """
    
    if pd.notna(x):
        month = int(str(x)[0:2])
        return calendar.month_name[month]
    else:
        return x

In [ ]:
# extract the birth month from the dob data
df['birth_month'] = df['dob'].apply(lambda x: birth_month_extractor(x))
# set any dob data which does not contain the year to nan
df.loc[df['dob'].str.len() < 10, 'dob'] = np.nan
# convert the dob column to datetime
df['dob'] = pd.to_datetime(df['dob'])
# drop the now-unnecessary '13' and '14' columns
df.drop(['13', '14'], axis=1, inplace=True)

In [32]:
# generate today's date
today = pd.to_datetime(str(date.today()))
# calculate the current age of users, based on their birthdate
df['age'] = df['dob'].apply(lambda x: (today - x)).dt.days // 365
# drop the dob column
df.drop('dob', axis=1, inplace=True)

# Export Cleaned Data

In [33]:
# export the cleaned data to a csv file, for subsequent import into Tableau
df.to_csv('data/facebook_leak.csv')